In [ ]:
#Word embeddings
import numpy as np
embeddings_index = {}
f = open('') #GloVe word embeddings
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, max_length_emb))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
def training5(X_test,X_test_emo, X_train,X_train_emo, y_train, X_val,X_val_emo,y_val, epocas=7):
  nlp_input = Input(shape=(max_length,), name='nlp_input')
  meta_input=Input(shape=(4,))
  emb_1 = Embedding(len(word_index)+1, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)(nlp_input)
  y=Conv1D(250, (3), kernel_regularizer=regularizers.l2(l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activation='relu')(emb_1)
  y=GlobalMaxPooling1D()(y)
  conc = concatenate([y, meta_input])
  y=Dense(254, kernel_regularizer=regularizers.l2(l2=1e-4),bias_regularizer=regularizers.l2(1e-4), activation='relu')(conc) #128
  output = Dense(1,kernel_regularizer=regularizers.l2(l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-4), activation='sigmoid')(y)
  model = Model(inputs=[nlp_input,meta_input], outputs=[output])
  opt = keras.optimizers.Adam(learning_rate=0.007)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  model.fit(x=[X_train,X_train_emo],y=y_train, epochs=epocas, validation_data=([X_val,X_val_emo],y_val),batch_size = 32, verbose = 1)
  y_pred=model.predict([X_test,X_test_emo])
  y_pred=np.around(y_pred)
  y_pred=y_pred.astype(int)
  return y_pred

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

def kParticiones_KM3(particiones,X,X_emo,y, epocas=10):
    metricas_k=pd.DataFrame(columns=['accuracy', 'precision', 'recall','f1-score'], index=(range(1,particiones+1)))
    #kf = KFold(n_splits=particiones, shuffle=True)
    kf = StratifiedKFold(n_splits=particiones)
    p = [];  r = []; a = []; funo = []; area=[]
    for train_index, test_index in kf.split(X,y):# K-PARTICIONES
        X_train=X[train_index] #
        X_train_emo=X_emo[train_index]
        test_x=X[test_index]
        test_x_emo=X_emo[test_index]
        y_train=y[train_index]
        test_y=y[test_index]
        X_test, X_val,X_test_emo, X_val_emo, y_test, y_val = train_test_split(test_x,test_x_emo, test_y, test_size=0.5)
        y_pred=training5(X_test,X_test_emo,X_train,X_train_emo, y_train,X_val,X_val_emo, y_val, epocas=epocas)
        pr,rec,acc,f1 = testing(y_pred, y_test) #Test the model
        p.append(pr) #Metrics
        r.append(rec)
        a.append(acc)
        funo.append(f1)
        m = tf.keras.metrics.AUC()#num_thresholds=3
        m.update_state(y_test, y_pred)
        ar=m.result().numpy()
        area.append(ar)
        m.reset_state()
    metricas_k['accuracy']=a; metricas_k['precision']=p;metricas_k['recall']=r;metricas_k['f1-score']=funo;
    precision=np.mean(p) #Metrics after validation
    recall=np.mean(r)
    accuracy=np.mean(a)
    f1_score=np.mean(funo)
    area_curva=np.mean(area)
    return precision,recall,accuracy,f1_score,area_curva,metricas_k